In [1]:
# Header block to include all modules that must be imported ahead of time
# Only needs to be run once per session, and each time a new module is added

import requests
import json
import pandas as pd # this will need to be installed via command line first
import lxml # this will need to be installed via command line first(as well)
import html5lib # this will need to be installed via command line first
import numpy as np
import re

In [2]:
# Method to retrieve all team names, confrences, and divisions 

def GetAllTeamNamesConfrencesDivisions():
    # Use requests to get the raw HTML response from overthecap.com
    CBSResponse = requests.get(f'https://www.cbssports.com/nfl/teams/')
    
    # If we get a normal response, proceed with data scraping
    if CBSResponse.status_code == 200:
        
        # Confirm the request was successful
        print(f"Request succeeded with status code {CBSResponse.status_code}")

        # Use Pandas to read the HTML content and put it into a DataFrame
        TableTag = 'TableBase-table'
        DataFrames = pd.read_html(CBSResponse.content, attrs={'class': TableTag})
        
        # Select the specific parts of the dataframe we want 
        TeamsDFAFC = DataFrames[0][['East']]
        TeamsDFNFC = DataFrames[1][['East']]
        
        # Insert a new row at the top with column name
        TeamsDFAFC.insert(0, 'Conference', 'AFC')
        TeamsDFNFC.insert(0, 'Conference', 'NFC')
        
        # Generate a list of divisions in the order the appear
        DivisionsList = ['East', TeamsDFAFC.iloc[4, 1], TeamsDFAFC.iloc[9, 1], TeamsDFAFC.iloc[14, 1]]
        
        # The CBS table includes division labels as rows so they need to be removed
        TeamsDFAFC.drop(labels = [4, 9, 14], inplace = True)
        TeamsDFNFC.drop(labels = [4, 9, 14], inplace = True)
        
        # Rename the column with all the teamnames, concatinate the dataframes together
        TeamsDFAFC.rename(columns = {'East': 'TeamName'}, inplace = True)
        TeamsDFNFC.rename(columns = {'East': 'TeamName'}, inplace = True)
        TeamsDF = pd.concat([TeamsDFAFC, TeamsDFNFC], ignore_index = True)
        
        # Create list to add each division to its respective team
        DivisionsFullList = []
        for i in range(2):
            for division in DivisionsList:
                for j in range(4):
                    DivisionsFullList.append(division)
        
        # Add the new column to the teams dataframe
        TeamsDF['Division'] = DivisionsFullList
        
        # Change the order of columns to match the database schema  
        new_order = ['TeamName','Conference', 'Division']
        TeamsDF = TeamsDF.loc[:, new_order]
        
        # Sort the teams dataframe by team name
        TeamsDF.sort_values(by = ['TeamName'], inplace = True)
        
        # Turn the DataFrame into a numpy array and sort it by team name
        TeamsArray = TeamsDF.to_numpy().tolist()
        
        return TeamsArray

    # If we don't get a normal reponse, stop scraping
    else:
        print(f"Request failed with status code {CBSResponse.status_code}")
        return -1


In [3]:
# Method to get all team abbreviations
def GetAllTeamAbbrvs():
   # Define the API endpoint and parameters
    endpoint = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "titles": "Wikipedia:WikiProject_National_Football_League/National_Football_League_team_abbreviations",
        "prop": "revisions",
        "rvprop": "content"
    }

    # Make the request to the Wikipedia API
    WikiResponse = requests.get(endpoint, params=params)
    
    # If we get a normal response, proceed with data scraping
    if WikiResponse.status_code == 200:
        
        # Confirm the request was successful
        print(f"Request succeeded with status code {WikiResponse.status_code}")
        
        # Extract the JSON data from the response
        WikiData = WikiResponse.json()
        PageContent = WikiData['query']['pages']['53669795']['revisions'][0]['*']
        
        # Sort the team abbreviations into a list, select only the abbreviations we want
        AllAbbeviations = re.findall(r'\b[A-Z]{2,3}\b', PageContent)
        TeamAbbrviations = AllAbbeviations[4::2] # This selects every other element in the list from index 4
        
        return TeamAbbrviations

    # If we don't get a normal reponse, stop scraping
    else:
        print("Error fetching data from the Wikipedia API.")
        return -1


In [4]:
# Method to contruct a list of all team names, abbreviations, confrences, divisions
# Creates the Complete Team List as a csv

def CreateCompleteTeamList():
    # Get all team names, confrences, and divisions
    TeamNamesConfrencesDivisions = GetAllTeamNamesConfrencesDivisions()
    
    # Get all team abbreviations
    TeamAbbrvs = GetAllTeamAbbrvs()
    
    # Concatinate the two lists
    CompleteTeamsList = []
    for i in range(len(TeamNamesConfrencesDivisions)):
        CompleteTeamsList.append([TeamNamesConfrencesDivisions[i][0], TeamAbbrvs[i], TeamNamesConfrencesDivisions[i][1], TeamNamesConfrencesDivisions[i][2]])

    # Create a dataframe from the list
    CompleteTeamsDF = pd.DataFrame(CompleteTeamsList)
    
    # Add the column names
    CompleteTeamsDF.columns = ['TeamName', 'TeamAbbrv', 'Conference', 'Division']
    print(CompleteTeamsDF)
    print("\n")
    
    # Create a .csv file with all the team names, abbreviations, confrences, and divisions
    csvPathName = f'./Teams-List.csv'
    pd.DataFrame(CompleteTeamsDF).to_csv(csvPathName, index = False, header = False)

    
    # Read the csv back for testing purposes
    TestDF = pd.read_csv(f'./Teams-List.csv', header = None)
    TestDF.columns = ['TeamName', 'TeamAbbrv', 'Conference', 'Division']
    print(TestDF)
    
CreateCompleteTeamList()

Request succeeded with status code 200


/tmp/ipykernel_1254/1493990831.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TeamsDFAFC.drop(labels = [4, 9, 14], inplace = True)
/tmp/ipykernel_1254/1493990831.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TeamsDFNFC.drop(labels = [4, 9, 14], inplace = True)
/tmp/ipykernel_1254/1493990831.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TeamsDFAFC.rename(columns = {'East': 'TeamName'}, inplace = True)
/tmp/ipykernel_

Request succeeded with status code 200
                 TeamName TeamAbbrv Conference Division
0       Arizona Cardinals       ARI        NFC     West
1         Atlanta Falcons       ATL        NFC    South
2        Baltimore Ravens       BAL        AFC    North
3           Buffalo Bills       BUF        AFC     East
4       Carolina Panthers       CAR        NFC    South
5           Chicago Bears       CHI        NFC    North
6      Cincinnati Bengals       CIN        AFC    North
7        Cleveland Browns       CLE        AFC    North
8          Dallas Cowboys       DAL        NFC     East
9          Denver Broncos       DEN        AFC     West
10          Detroit Lions       DET        NFC    North
11      Green Bay Packers        GB        NFC    North
12         Houston Texans       HOU        AFC    South
13     Indianapolis Colts       IND        AFC    South
14   Jacksonville Jaguars       JAC        AFC    South
15     Kansas City Chiefs        KC        AFC     West
16      L

In [5]:
# Method to get the salary cap hits for every player currently signed on an NFL team
# Creates a csv with columns player name, salary cap hit, and team location abbrviation 

def GetTeamSalaryCapHits(team):
    # Use requests to get the raw HTML response from overthecap.com
    OTCResponse = requests.get(f'https://overthecap.com/salary-cap/{team[0]}')
        
    # If we get a normal response proceed with data scraping
    if OTCResponse.status_code == 200:
        print(f"Request succeeded with status code {OTCResponse.status_code}")
        TeamNameCaptalized = (team[0].replace("-", " ")).title()
        print(f"Here are the Salary Cap hits for all players who are currently signed with the {TeamNameCaptalized}:")

        # Pandas read_html method allows the table contents to be put into a DataFrame
        TableTag = 'salary-cap-table contracted-players'
        DataFrames = pd.read_html(OTCResponse.content, attrs = {'class': TableTag})
        
        # Select the specific parts of the dataframe we want 
        SalariesDF = DataFrames[0][['Player', 'Cap Number']]
        if len(SalariesDF) >= 51:
            SalariesDF = SalariesDF.drop(51, axis=0)
        # Add team location abv to each player
        SalariesDF = SalariesDF.assign(TeamLOC = team[1])
        SalariesDF = SalariesDF.assign(TeamName = (team[0].replace("-", " ")).title())
        
        
        # Change the order of columns to make more sense semantically 
        new_order = ['Player', 'TeamLOC', 'TeamName', 'Cap Number']
        SalariesDF = SalariesDF.loc[:, new_order]
        
        # Specify filename and path
        csvPathName = f'./salary-Lists/{TeamNameCaptalized}-Player-Salary-List.csv'
        
        # Create a .csv file with all the player names and their cap hits
        SalariesDF.to_csv(csvPathName, index = False, header = False)
        
        # Read the csv back for testing purposes
        SalariesCSV = pd.read_csv(f'./salary-Lists/{TeamNameCaptalized}-Player-Salary-List.csv', header = None)
        SalariesCSV.columns = ['Player', 'TeamLOC', 'TeamName', 'Cap Number']
        print(SalariesCSV)
        
    # If we don't get a normal reponse, stop scraping
    else:
        print(f"Request failed with status code {OTCResponse.status_code}")
        return -1


In [6]:
# A list of links to Over The Cap Salary Cap pages for various teams
NFLTeams = [['buffalo-bills', 'BUF'], ['miami-dolphins', 'MIA'], ['new-england-patriots', 'NE'], ['new-york-jets', 'NYJ'],
            ['baltimore-ravens', 'BAL'], ['cincinnati-bengals', 'CIN'], ['cleveland-browns', 'CLE'], ['pittsburgh-steelers', 'PIT'],
            ['houston-texans', 'HOU'], ['indianapolis-colts', 'IND'], ['jacksonville-jaguars', 'JAX'], ['tennessee-titans', 'TEN'], 
            ['denver-broncos', 'DEN'], ['kansas-city-chiefs', 'KC'], ['las-vegas-raiders', 'LV'], ['los-angeles-chargers', 'LAC'],
            ['dallas-cowboys', 'DAL'], ['new-york-giants', 'NYG'], ['philadelphia-eagles', 'PHI'], ['washington-commanders', 'WSH'],
            ['chicago-bears', 'CHI'], ['detroit-lions', 'DET'], ['green-bay-packers', 'GB'], ['minnesota-vikings', 'MIN'],
            ['atlanta-falcons', 'ATL'], ['carolina-panthers', 'CAR'], ['new-orleans-saints', 'NO'], ['tampa-bay-buccaneers', 'TB'],
            ['arizona-cardinals', 'ARI'], ['los-angeles-rams', 'LAR'], ['san-francisco-49ers', 'SF'], ['seattle-seahawks', 'SEA']
           ]

# Call the method defined above to get all the player data
def GetMultipleTeamSalaryCapHits(TeamList):
    for team in TeamList:
        GetTeamSalaryCapHits(team)

# GetMultipleTeamSalaryCapHits(NFLTeams)
GetTeamSalaryCapHits(['san-francisco-49ers', 'SF'])

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the San Francisco 49Ers:
             Player TeamLOC             TeamName   Cap Number
0    Trent Williams      SF  San Francisco 49Ers  $31,568,542
1      Deebo Samuel      SF  San Francisco 49Ers  $28,633,765
2   Charvarius Ward      SF  San Francisco 49Ers  $18,401,000
3         Nick Bosa      SF  San Francisco 49Ers  $14,671,000
4     Brandon Aiyuk      SF  San Francisco 49Ers  $14,124,000
..              ...     ...                  ...          ...
86   Ricky Pearsall      SF  San Francisco 49Ers     $795,000
87   Tayler Hawkins      SF  San Francisco 49Ers     $795,000
88    Isaac Alarcon      SF  San Francisco 49Ers     $795,000
89     Shakel Brown      SF  San Francisco 49Ers     $795,000
90       Tay Martin      SF  San Francisco 49Ers     $795,000

[91 rows x 4 columns]
